In [1]:

# #@title Imports and Utils { display-mode: "form" }

# %matplotlib inline
# %config InlineBackend.figure_format = 'svg'
# %config InlineBackend.rc = {'figure.figsize': (10.0, 10.0)}

# !git clone -qq https://github.com/Lin-Sinorodin/SwinIR_wrapper.git
# !pip install -qq timm
from SwinIR_wrapper.SwinIR_wrapper import SwinIR_SR

import cv2
import torch
import urllib.request
import matplotlib.pyplot as plt

device = torch.device('cuda: 1' if torch.cuda.is_available() else 'cpu')

if torch.cuda.is_available():
    print(f'Using GPU: {torch.cuda.get_device_properties(0).name}')
else:
    print('Using CPU. Concider using GPU for faster inference.')

def compare_sr_with_original(img_lq, img_hq):
    plt.figure()

    plt.subplot(1, 2, 1)
    plt.imshow(img_lq[::,::,::-1])
    plt.title(f'Original - {img_lq.shape}')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(img_hq[::,::,::-1])
    plt.title(f'Super Resolution - {img_hq.shape}')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/different_types_gans/SwinIR_wrapper/SwinIR_wrapper/SwinIR.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Using GPU: NVIDIA A100-SXM4-80GB


In [2]:

# #@title Get Test Image { run: "auto", display-mode: "form" }

# #@markdown > #### Choose between uploading image or downloading from url:
# get_image_method = "download from url" #@param ["upload file", "download from url"]

# #@markdown * for uploaded image, write it's path:
# uploaded_file = "/content/my_uploaded_image.jpeg" #@param {type:"string"}
# if get_image_method == 'upload file':
#     path = uploaded_file

# #@markdown * for url download, provide the image url:
# url = 'https://map.pepperdine.edu/maps/UMAP_2011012422072_6_8_19.jpg' #@param {type:"string"}
# if get_image_method == 'download from url':
#     path = url.split('/')[-1]
#     urllib.request.urlretrieve(url, path)

In [3]:
path="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/vlm_data/bihar_most_15/images/9744500_2967894.tif"

In [4]:


#@title Setup Super Resolution Model { run: "auto" }
pretrained_model = "real_sr x4" #@param ["real_sr x4", "classical_sr x2", "classical_sr x3", "classical_sr x4", "classical_sr x8", "lightweight x2", "lightweight x3", "lightweight x4"]

model_type, scale = pretrained_model.split(' ')
scale = int(scale[1])

# initialize super resolution model
sr = SwinIR_SR(model_type, scale)

print(f'Loaded {pretrained_model} successfully')

/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/different_types_gans/SwinIR_wrapper/SwinIR_wrapper/SwinIR.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pret

Loaded real_sr x4 successfully


In [5]:
# load the test image from previus step
img_lq = cv2.imread(path, cv2.IMREAD_COLOR)

# feed the image to the SR model
img_hq = sr.upscale(img_lq)    
image_resized = cv2.resize(img_hq, (img_lq.shape[1], img_lq.shape[0]), interpolation = cv2.INTER_AREA)

# save the result
cv2.imwrite(f'{pretrained_model}.jpg', image_resized)
# show results
compare_sr_with_original(img_lq, image_resized) 

[ WARN:0@12.274] global loadsave.cpp:241 findDecoder imread_('/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/data/vlm_data/bihar_most_15/images/9744500_2967894.tif'): can't open/read file: check file path/integrity


AttributeError: 'NoneType' object has no attribute 'shape'

In [6]:
import os
import cv2
 # Replace with the actual module or model you're using

# Paths
input_dir = '../data/region_performance/bihar_same_class_count_10_120_1000/images'
output_dir = '../data/swinir_data/bihar_same_class_count_10_120_1000/images'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Function to process images
def process_images(input_dir, output_dir):
    for filename in os.listdir(input_dir):
        if filename.endswith('.tif'):  # Check for .tif files
            # Load the low-quality image
            img_lq_path = os.path.join(input_dir, filename)
            img_lq = cv2.imread(img_lq_path, cv2.IMREAD_COLOR)
            
            # Apply the super-resolution model
            img_hq = sr.upscale(img_lq)  # Adjust the method based on your SR model's API
            
            # Resize the image back to the original dimensions
            image_resized = cv2.resize(img_hq, (img_lq.shape[1], img_lq.shape[0]), interpolation=cv2.INTER_AREA)
            
            # Save the result as a .png file
            output_filename = os.path.splitext(filename)[0] + '.png'
            output_path = os.path.join(output_dir, output_filename)
            cv2.imwrite(output_path, image_resized)
            
            # Optional: Display or compare the images (for debugging or validation)
            # compare_sr_with_original(img_lq, image_resized)  # Implement or call your comparison function

# Execute the function
process_images(input_dir, output_dir)
